## Introduction

Is is possible to predict the gender of a character in a story using the pronouns around them?

Yes

He and she are objective pronouns

Him and her are subjective pronouns

His and her are possessive adjectives

His and hers are possessive pronouns

"Her" is possessive when followed by a noun.

## Methodology

## Hypothesis



In [ ]:
%pip install sparknlp
%pip install pyspark
%pip install spacy
import sparknlp
import os
import spacy
import pyspark
from pyspark.sql import SparkSession
from tqdm import tqdm
spark = sparknlp.start()

In [ ]:
from sparknlp.pretrained import PretrainedPipeline
root = os.path.dirname(os.path.realpath('sparknlp.ipynb'))
file = open(root + '/pg1342.txt',"r")
pride = file.read()

In [ ]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


In [ ]:
#result = pipeline.annotate(pride)

In [ ]:
tok_tag = [(result['token'], result['pos'])]
zips = [list(zip(tt[0],tt[1])) for tt in tok_tag]

In [ ]:
pronouns = ['he','him','his','she','her','hers']
def pronoun_check(x):
  '''
  x = list of tuples
  returns list of tuples with only pronouns
  '''
  pro_toks = []
  male_count, female_count = 0, 0
  for i in x:
    flag = False
    last_pos = ''
    last_tok = ''
    for y in i:
      if flag:
        if pos != 'NN' and pos != 'NNS' and token == '.':
          female_count += 1
          flag = False
          continue
      token, pos = y
      token = token.lower()
      # Magic (identifies instances of 'her' that are objective)
      if token in pronouns:
        if token == 'her':
          flag = True
          last_pos = pos
          last_tok = token
        else:
            if token == 'he' or token == 'him':
              male_count += 1
            else:
              female_count += 1

  return male_count, female_count

#def count_pronouns(x):

In [ ]:
class Node:
  def __init__(self, data):
    # reference to next node in DLL
    self.next = None
    # reference to previous node in DLL
    self.prev = None
    self.data = data

def insert_at_end(head, data):
  # insert new node at the end of the list
  new_node = Node(data)
  if head is None:
    return new_node

  current = head
  while current.next:
    current = current.next

  current.next = new_node
  new_node.prev = current
  return head

def display(head):
  current = head
  while current:
    print(current.data, end = " <-> ")
    current = current.next
  print(None)

In [ ]:
head = None
head = insert_at_end(head, 1)
head = insert_at_end(head, 2)
head = insert_at_end(head, 3)

print("Doubly Linked List after insertion at the end:")
display(head)

Doubly Linked List after insertion at the end:
1 <-> 2 <-> 3 <-> None


In [ ]:
pride = pride.replace('Mr.','Mr')
pride = pride.replace('Mrs.','Mrs')
pride = pride.replace('Ms.','Ms')
pride = pride.replace('Dr.','Dr')
pride = pride.replace('_','')
pride = pride.replace('-','')
pridesplit = pride.split('.')

In [ ]:
def period(sentences):
  sent_list = []
  for sent in sentences:
    sent = sent+'.'
    sent_list.append(sent)
  return sent_list

In [ ]:
pridesplit = period(pridesplit)

In [ ]:
for sent in pridesplit:
  results = []
  result = pipeline.annotate(sent)
  results.append(result)
return results

In [ ]:
nlp = spacy.load('en_core_web_sm')
personGenderCount = {} # person: malePronounCount, femalePronounCount
# for sent in pridesplit:
for sent in tqdm(pridesplit[:1000]):
  doc = nlp(sent)
  people = []
  for ent in doc.ents:
    if ent.label_ == 'PERSON':
      people.append(ent.text)
  if len(people) == 0:
    continue
  annoted_sent = pipeline.annotate(sent)
  tok_tag = [(annoted_sent['token'], annoted_sent['pos'])]
  zips = [list(zip(tt[0],tt[1])) for tt in tok_tag]
  male_count, female_count = pronoun_check(zips)
  for person in people:
    try:
      person_male_count = personGenderCount[person][0]
      person_female_count = personGenderCount[person][1]
      personGenderCount[person] = person_male_count + male_count, person_female_count + female_count
    except KeyError:
      personGenderCount[person] = male_count, female_count


personGenderCount


  1%|▏         | 14/1000 [00:02<02:11,  7.51it/s]